# 🤖 03 – Modelling & Evaluation

**Objective:**  
Train, evaluate, and interpret a baseline machine learning model to predict whether a Netflix title is a potential “hit” based on its metadata features.

**Scope of this notebook:**
- Load the processed dataset from **`netflix_clean`** (created in Notebook 02)
- Select relevant numeric and categorical features for modelling  
- Split the dataset into training and test sets  
- Build and train a baseline classification model (Random Forest or Logistic Regression)  
- Evaluate model performance using key metrics:
  - Accuracy, Precision, Recall, F1-score, ROC-AUC  
- Log model parameters and metrics using **MLflow** for experiment tracking  
- Analyse feature importance and interpretability  
- Save the trained model for downstream use (e.g., predictions or dashboard integration)

**Outcome:**  
A reproducible, MLflow-tracked baseline model capable of classifying titles as *Hit* or *Non-Hit*, with performance metrics and feature insights ready for visualisation and reporting in Notebook 04.

In [0]:
from pyspark.sql import functions as F

# Load the cleaned dataset created in Notebook 02
df_spark = spark.table("netflix_clean")

print(f"Row count: {df_spark.count():,}")
print("Columns:", df_spark.columns)

display(df_spark.limit(5))

## Define features and label

We’ll use:
	•	Label: is_hit
	•	Numeric features: release_year, duration_num, is_movie
	•	Categorical features: category, rating, country

In [0]:
# Define feature and label columns
label_col = "is_hit"

numeric_features = ["release_year", "duration_num", "is_movie"]
categorical_features = ["category", "rating", "country"]

feature_cols = numeric_features + categorical_features

# Drop rows with missing label or feature values
df_spark_model = df_spark.dropna(subset=[label_col] + feature_cols)

print(f"Row count after dropping nulls: {df_spark_model.count():,}")

In [0]:
display(
    df_spark_model.groupBy(label_col).count().orderBy(label_col)
)

In [0]:
# Convert to Pandas for scikit-learn
df_model = df_spark_model.select(feature_cols + [label_col]).toPandas()

print(df_model.shape)
df_model.head()

In [0]:
from sklearn.model_selection import train_test_split

X = df_model[feature_cols]
y = df_model[label_col]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

## Build pipeline: preprocessing + RandomForest

We will use a ColumnTransformer for numeric/categorical handling and a RandomForestClassifier as the baseline model.


In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Preprocessing: scale numeric, one-hot encode categorical
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

# Baseline model
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42,
    n_jobs=-1
)

# Full pipeline
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", rf_clf),
])

# Train
model.fit(X_train, y_train)

## Evaluate model performance

In [0]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    ConfusionMatrixDisplay
)
import matplotlib.pyplot as plt

# Predictions
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy : {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall   : {rec:.3f}")
print(f"F1-score : {f1:.3f}")
print(f"ROC-AUC  : {roc_auc:.3f}\n")

print("Classification report:\n")
print(classification_report(y_test, y_pred, zero_division=0))

# Confusion matrix plot
disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title("Confusion Matrix")
plt.show()

## Feature importance (top drivers)

We’ll grab the feature names after preprocessing and show the top contributors.

In [0]:
import numpy as np
import pandas as pd

# Get feature names after preprocessing
ohe = model.named_steps["preprocessor"].named_transformers_["cat"]
encoded_cat_features = ohe.get_feature_names_out(categorical_features)

all_features = numeric_features + list(encoded_cat_features)

importances = model.named_steps["classifier"].feature_importances_

feat_imp = pd.DataFrame({
    "feature": all_features,
    "importance": importances
}).sort_values("importance", ascending=False)

feat_imp.head(15)

feat_imp.head(15).plot(
    x="feature",
    y="importance",
    kind="barh",
    figsize=(8, 6)
)
plt.gca().invert_yaxis()
plt.title("Top 15 Feature Importances")
plt.tight_layout()
plt.show()

## 📈 Log Model & Metrics to MLflow

**Purpose:**  
Track our experiment metadata, metrics, and model artefacts using MLflow — Databricks’ built-in experiment tracking and model registry framework.

**What this section does:**
- Creates (or connects to) an MLflow experiment named `/Shared/StreamSense_Experiments`
- Logs key model parameters (`n_estimators`, `max_depth`)
- Logs performance metrics (accuracy, precision, recall, F1, ROC-AUC)
- Captures the model’s input schema and a small example for reproducibility
- Saves the trained scikit-learn pipeline as a versioned model artefact

This ensures that each model training run is fully traceable — including configuration, metrics, and code version — enabling easy comparison, rollback, and future deployment.

In [0]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

# Set or create an experiment; adjust path if you want it elsewhere
mlflow.set_experiment("/Shared/StreamSense_Experiments")

with mlflow.start_run(run_name="baseline_random_forest"):
    # Log model parameters and metrics
    mlflow.log_param("n_estimators", rf_clf.n_estimators)
    mlflow.log_param("max_depth", rf_clf.max_depth)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", prec)
    mlflow.log_metric("recall", rec)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("roc_auc", roc_auc)

    # Add model signature and input example for reproducibility
    # Cast numeric columns to float to avoid MLflow integer schema warnings
    X_train = X_train.astype({
        "release_year": "float64",
        "duration_num": "float64",
        "is_movie": "float64"
    })
    input_example = X_train.head(1)
    signature = infer_signature(X_train, model.predict(X_train))

    # Log model with metadata
    mlflow.sklearn.log_model(
        model,
        "model",
        signature=signature,
        input_example=input_example
    )

> 💡 **Note:**  
> MLflow may issue a warning when logging models with integer features that could contain missing values in the future.  
> To avoid schema mismatches, we explicitly cast numeric columns (`release_year`, `duration_num`, `is_movie`) to `float64` before inferring the model signature.  
> This ensures robust schema enforcement during future predictions or model serving.

## 🧾 Notebook Summary – 03_Modelling_And_Evaluation

**Objective:**  
Train and evaluate a baseline classification model to predict whether a Netflix title is a potential "hit" using engineered metadata features.

**Key steps completed:**
- Loaded the cleaned dataset from the **`netflix_clean`** Delta table
- Selected numeric (`release_year`, `duration_num`, `is_movie`) and categorical (`category`, `rating`, `country`) features
- Split the data into training and test sets with stratified sampling
- Built a scikit-learn pipeline with:
  - StandardScaler for numeric features
  - OneHotEncoder for categorical features
  - RandomForestClassifier as the baseline model
- Evaluated the model using accuracy, precision, recall, F1-score, and ROC-AUC
- Inspected top feature importances to understand drivers of the `is_hit` prediction
- Optionally logged parameters, metrics, and the model artifact in **MLflow**

**Next steps:**
- Use this model in **Notebook 04** to:
  - Build a "What-if" prediction interface for hypothetical titles
  - Create visualisations and dashboards to share insights
  - Refine the model and features based on performance and business interpretability